# VNStock Backtrader Bot (Modular)
- Structure: data ? features ? strategy ? backtest ? alerts, with stubs for execution/AI.
- Requirements: `vnstock`, `pandas`, `pandas_ta`, `backtrader`, `matplotlib`, `requests`.
- Configure ticker/dates/env in the config cell.


In [ ]:
# Install deps (run once per environment)
import sys, subprocess
pkgs = ['vnstock', 'pandas', 'pandas_ta', 'backtrader', 'matplotlib', 'requests']
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', '--upgrade'] + pkgs)
print('Done')


In [ ]:
import os, datetime as dt
import pandas as pd
import pandas_ta as ta
import backtrader as bt
import matplotlib.pyplot as plt
import requests
from vnstock import Vnstock

pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 160)

# Config
CFG = {
    'ticker': 'FPT',
    'start': '2023-01-01',
    'end': dt.date.today().isoformat(),
    'alert_thresh_pct': 2.0,
    'cash': 100000.0,
    'commission': 0.001,
}

# Telegram (optional)
TELEGRAM_BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')
TELEGRAM_CHAT_ID = os.getenv('TELEGRAM_CHAT_ID')
print('Config loaded', CFG)


## 1) Data module

In [ ]:
def fetch_history(symbol: str, start: str, end: str) -> pd.DataFrame:
    client = Vnstock()
    stock = client.stock(symbol=symbol)
    df = stock.quote.history(start=start, end=end)
    df = df.rename(columns=str.lower)
    df['date'] = pd.to_datetime(df['time'])
    df = df[['date','open','high','low','close','volume']].sort_values('date').reset_index(drop=True)
    return df

raw = fetch_history(CFG['ticker'], CFG['start'], CFG['end'])
raw.tail()


## 2) Features/Indicators module

In [ ]:
def process_data(df: pd.DataFrame) -> pd.DataFrame:
    out = df.dropna().copy()
    out['return'] = out['close'].pct_change().fillna(0)
    return out

def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['sma_fast'] = ta.sma(out['close'], length=20)
    out['sma_slow'] = ta.sma(out['close'], length=50)
    out['rsi'] = ta.rsi(out['close'], length=14)
    out['atr'] = ta.atr(out['high'], out['low'], out['close'], length=14)
    out['obv'] = ta.obv(out['close'], out['volume'])
    return out

data = process_data(raw)
feat = add_indicators(data)
feat.tail()


## 3) Strategy module (Backtrader)

In [ ]:
class SmaRsiStrategy(bt.Strategy):
    params = dict(fast=20, slow=50, rsi_period=14, rsi_long=50, rsi_short=50)

    def __init__(self):
        self.sma_fast = bt.ind.SMA(self.data.close, period=self.p.fast)
        self.sma_slow = bt.ind.SMA(self.data.close, period=self.p.slow)
        self.rsi = bt.ind.RSI(self.data.close, period=self.p.rsi_period)
        self.crossover = bt.ind.CrossOver(self.sma_fast, self.sma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0 and self.rsi > self.p.rsi_long:
                self.buy()
            elif self.crossover < 0 and self.rsi < self.p.rsi_short:
                self.sell()
        else:
            if self.position.size > 0 and self.crossover < 0:
                self.close()
            if self.position.size < 0 and self.crossover > 0:
                self.close()


## 4) Backtest runner

In [ ]:
def run_backtest(df: pd.DataFrame):
    bt_df = df[['date','open','high','low','close','volume']].dropna().copy()
    bt_df = bt_df.set_index('date')
    feed = bt.feeds.PandasData(dataname=bt_df)
    cerebro = bt.Cerebro()
    cerebro.adddata(feed)
    cerebro.addstrategy(SmaRsiStrategy)
    cerebro.broker.set_cash(CFG['cash'])
    cerebro.broker.setcommission(CFG['commission'])
    start_val = cerebro.broker.getvalue()
    cerebro.run()
    end_val = cerebro.broker.getvalue()
    ret = end_val / start_val - 1
    return {'start': start_val, 'end': end_val, 'return': ret}

bt_stats = run_backtest(feat)
bt_stats


## 5) Visualization (price + SMAs)

In [ ]:
def plot_price(df: pd.DataFrame, title='Price with SMAs'):
    fig = plt.figure(figsize=(10,4))
    plt.plot(df['date'], df['close'], label='Close')
    plt.plot(df['date'], df['sma_fast'], label='SMA20', alpha=0.7)
    plt.plot(df['date'], df['sma_slow'], label='SMA50', alpha=0.7)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_price(feat)


## 6) Alerts (Telegram)

In [ ]:
def send_telegram(message: str):
    if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_ID:
        print('Telegram env vars not set; skipping')
        return
    url = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage'
    resp = requests.post(url, json={'chat_id': TELEGRAM_CHAT_ID, 'text': message})
    if resp.ok:
        print('Sent')
    else:
        print('Failed', resp.status_code, resp.text)

last = feat.iloc[-1]
prev = feat.iloc[-2]
pct = (last['close'] / prev['close'] - 1) * 100
msg = f"{CFG['ticker']} move {pct:.2f}% | close={last['close']:.2f} | backtest return={bt_stats['return']:.2%}"
if abs(pct) >= CFG['alert_thresh_pct']:
    send_telegram(msg)
else:
    print('No alert triggered')


## 7) Execution adapter (stub for future auto-trade)

In [ ]:
class BrokerAdapter:
    # Stub for future broker/exchange API
    def __init__(self, name='paper'):
        self.name = name

    def place_order(self, side: str, qty: float, price: float=None):
        print(f'[Broker:{self.name}] order side={side} qty={qty} price={price}')

    def cancel_all(self):
        print(f'[Broker:{self.name}] cancel all')

broker = BrokerAdapter()
broker.place_order('buy', 100)


## 8) AI/ML placeholder (future)

In [ ]:
# Future: feature selection / ML model
# - Build feature matrix X, target y (e.g., next-day direction or return)
# - Train with time-series splits to avoid leakage
# - Export signal function that maps latest features to a trade decision
